In [1]:
import networkx as nx
import numpy as np

In [2]:
def random_walk_with_restart(G, s, e, restart_prob=0.3, max_iter=100):
    # Initialize probabilities
    nodes = list(G.nodes())
    prob = {node: 0 for node in nodes}
    prob[s] = 1

    for i in range(max_iter):
        next_prob = {node: 0 for node in nodes}
        
        # Random walk step
        for node in prob:
            if node in G:
                neighbors = list(G.neighbors(node))
                if neighbors:
                    shared_prob = (1 - restart_prob) * prob[node] / len(neighbors)
                    for neighbor in neighbors:
                        next_prob[neighbor] += shared_prob
        
        # Restart step
        next_prob[s] += restart_prob * prob[s]

        # Update probabilities
        prob = next_prob

        # Check for convergence (optional, for performance)
        if np.allclose(list(prob.values()), list(next_prob.values()), atol=1e-6):
            break

    return round(prob[e], 2)


In [3]:
# Define the edge index and create the graph
edge_index = [
    [ 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14, 15, 16 ],
    [ 1, 5, 0, 2, 1, 3, 2, 4, 9, 3, 5, 6, 0, 4, 4, 7, 6, 8, 7, 9, 13, 3, 8, 10, 9, 11, 10, 12, 11, 13, 14, 8, 12, 12, 15, 16, 14, 14 ]
]

G = nx.Graph()
for start, end in zip(*edge_index):
    G.add_edge(start, end)

# Define the node features (not used in RWR but given for context)
node_features = [
    [[ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 0, 1, 0, 0, 0, 0, 0 ],
    [ 0, 0, 1, 0, 0, 0, 0 ],
    [ 0, 0, 1, 0, 0, 0, 0 ],]
]

# Applying RWR for all pairs of nodes
num_nodes = len(G.nodes())
rwr_matrix = np.zeros((num_nodes, num_nodes))

for s in range(num_nodes):
    for e in range(num_nodes):
        rwr_matrix[s, e] = random_walk_with_restart(G, s, e)

rwr_matrix


array([[0.3 , 0.35, 0.  , 0.  , 0.  , 0.35, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.35, 0.3 , 0.35, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.35, 0.3 , 0.35, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.23, 0.3 , 0.23, 0.  , 0.  , 0.  , 0.  , 0.23, 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.23, 0.3 , 0.23, 0.23, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.35, 0.  , 0.  , 0.  , 0.35, 0.3 , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.35, 0.  , 0.3 , 0.35, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.35, 0.3 , 0.35, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  

In [25]:
def random_walk_with_restart_max_steps(G, s, e, max_steps=G.number_of_nodes()//2, max_iter=100):
    nodes = list(G.nodes())
    prob = {node: 0 for node in nodes}
    prob[s] = 1

    for _ in range(max_iter):
        next_prob = {node: 0 for node in nodes}
        step_counter = 0

        for node in prob:
            if node in G:
                # limit step by K max_steps
                # only consider valid steps
                if step_counter < max_steps:
                    # find neighbors
                    neighbors = list(G.neighbors(node))
                    if neighbors:
                        shared_prob = prob[node] / len(neighbors)
                        for neighbor in neighbors:
                            next_prob[neighbor] += shared_prob
                    step_counter += 1
                else:
                    # Restart from the starting node
                    next_prob[s] = 1
                    step_counter = 0

        prob = next_prob

        if np.allclose(list(prob.values()), list(next_prob.values()), atol=1e-6):
            break

    return round(prob[e],1)


In [23]:
nodes = list(G.nodes())
prob = {node: 0 for node in nodes}
prob

{0: 0,
 1: 0,
 5: 0,
 2: 0,
 3: 0,
 4: 0,
 9: 0,
 6: 0,
 7: 0,
 8: 0,
 13: 0,
 10: 0,
 11: 0,
 12: 0,
 14: 0,
 15: 0,
 16: 0}

In [36]:
list(G.neighbors(0))

[1, 5]

In [14]:
# Define the edge index and create the graph
edge_index = [
    [ 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14, 15, 16 ],
    [ 1, 5, 0, 2, 1, 3, 2, 4, 9, 3, 5, 6, 0, 4, 4, 7, 6, 8, 7, 9, 13, 3, 8, 10, 9, 11, 10, 12, 11, 13, 14, 8, 12, 12, 15, 16, 14, 14 ]
]

G = nx.Graph()
for start, end in zip(*edge_index):
    G.add_edge(start, end)

# Define the node features (not used in RWR but given for context)
node_features = [
    [[ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 1, 0, 0, 0, 0, 0, 0 ],
    [ 0, 1, 0, 0, 0, 0, 0 ],
    [ 0, 0, 1, 0, 0, 0, 0 ],
    [ 0, 0, 1, 0, 0, 0, 0 ],]
]

# Applying RWR for all pairs of nodes
num_nodes = len(G.nodes())
rwr_matrix = np.zeros((num_nodes, num_nodes))

for s in range(num_nodes):
    for e in range(num_nodes):
        rwr_matrix[s, e] = random_walk_with_restart_max_steps(G, s, e)

rwr_matrix


array([[1. , 0.5, 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0. , 0. , 0.3, 1. , 0.3, 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.3, 1. , 0.3, 0.3, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0. , 0.5, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0. , 1. , 0.5, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 1. , 0.3, 0. , 0. , 0. ,
        0.3, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0.3, 1. , 0.3, 0. , 0. ,
        0

In [ ]:
[[1. , 0.5, 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0.3, 1. , 0.3, 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0.3, 1. , 0.3, 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0.5, 0. , 0. , 0. , 0.5, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0.5, 0. , 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 1. , 0.3, 0. , 0. , 0. , 0.3, 0. , 0. , 0. ],
 [0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0.3, 1. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 1. , 0.3, 0.3, 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 1. , 0.3, 0.3],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 1. ]]

In [12]:
G.number_of_nodes()

17